In [2]:
from langchain.vectorstores import chroma
from langchain.chat_models import ChatOpenAI

In [3]:
llm = ChatOpenAI()

In [4]:
llm.predict("hi")

'Hello! How can I assist you today?'

In [5]:
from langchain.document_loaders import TextLoader
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

# Load the document, split it into chunks, embed each chunk and load it into the vector store.
raw_documents = TextLoader('sotu.txt', encoding='utf-8').load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
documents = text_splitter.split_documents(raw_documents)

In [6]:
len(documents)

42

In [7]:
from langchain.schema import HumanMessage
from langchain.prompts import ChatPromptTemplate

import re

system_message = "What questions does this document answer?, Each question should start from a new line and the questions should be specific and not too vague. \n Document: "
text = "{document}"
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", system_message),
    ("human", text)
])
messages = [HumanMessage(content=text)]
question_docs = {}
for ind, document in enumerate(documents):
    if ind % 10 == 0:
        print(ind)
    message = chat_prompt.format_messages(document=document)
    output = llm(message)
    for question in output.content.split("\n"):
        # remove special characters using regex
        question = re.sub(r'[^a-zA-Z0-9\s]', '', question)     
        # remove leading numbers
        question = re.sub(r'^\d+\s+', '', question)   
        question_docs[question] = document.page_content

0
10
20
30
40


In [ ]:
# write the questions and the corresponding document to a json file
import json
with open('questions.json', 'w') as fp:
    json.dump(question_docs, fp)


In [6]:
# Define the LLM model
# Define the vector store
vector_store = chroma.VectorStore()

# Define a function to retrieve the top-k relevant documents
def retrieve_documents(query, k=5):
    # Compute the similarity between the query and all embedding questions in the vector store
    similarities = vector_store.similarity(query)
    
    # Retrieve the document chunks that correspond to the top-k relevant questions
    top_k_questions = similarities.argsort()[-k:][::-1]
    top_k_documents = [vector_store.get_document(question) for question in top_k_questions]
    
    return top_k_documents

# Define a function to add a document chunk to the vector store
def add_document(document_chunk):
    # Ask the LLM "What questions can this document answer?"
    questions = llm.generate(document_chunk, num_return_sequences=5)
    
    # Embed the question and document chunk into the vector store
    for question in questions:
        embedding = langchain.embed(question + document_chunk)
        vector_store.add_document(embedding, document_chunk)

TypeError: Can't instantiate abstract class VectorStore with abstract methods add_texts, from_texts, similarity_search